In [1]:
import psycopg2
import pandas as pd
import numpy as np
import os

## Exploramos tabla históricos "HIS_Elemento_Posiciones" y La de los Elementos "B_Elementos"

In [11]:
# cargar toda la tabla B_Elementos
conexion = psycopg2.connect(
    host="srvbu-bd",
    port="5432",
    database="BITAL_SACYR_ICONICA",
    user="adminpg",
    password="KRZEy5fxQyNmmVjeUseQ",
)

cursor = conexion.cursor()

cursor.execute('SELECT * FROM public."B_Elementos" WHERE "Fo_Tipoelemento" = 5;')
# Obtener todos los registros
registros = cursor.fetchall()

# crear un df con pandas
columnas = [desc[0] for desc in cursor.description]
df_elementos = pd.DataFrame(registros, columns=columnas)
display(df_elementos.head(20))

# Filtramos los nombres en M-108-0, M-139-0, M-43-0
df_elementos = df_elementos[
    df_elementos["Nombre"].isin(["M-108-0", "M-139-0", "M-43-0"])
]

# seleccionamos los pares Nombre, Id
for nombre, Id in zip(df_elementos["Nombre"], df_elementos["Id"]):
    print(f'"{nombre}": {Id},')


display(df_elementos)

,Id,Nombre,Fo_Tipoelemento,FechaBaja,TR_Coordenadas,TR_Ocupacion,TR_IntensidadCarrilAsc,TR_VelMedia,TR_Fecha,TR_Sentido_EsAsce,TR_Carriles,TR_IntensidadCarrilDesc,PuntoKm
0,6,Espira de A3,5,None,None,None,NaN,None,None,False,NaN,NaN,None
1,7,Espira de A3 prima,5,None,None,None,NaN,None,None,False,NaN,NaN,None
2,15,M-44-0,5,None,0101000020E6100000CA198A3BDE2C444077A04E797463...,0.00,33.0,None,None,False,NaN,33.0,None
3,16,M-267-0,5,None,0101000020E6100000CE531D72332A444040A374E95F32...,0.10,44.0,None,None,False,NaN,44.0,None
4,17,M-45-0,5,None,0101000020E61000000AF4893C492844401442075DC201...,0.20,55.0,None,None,False,NaN,55.0,None
5,18,M-109-0,5,None,0101000020E61000003E3C4B901126444064744012F6CD...,0.30,66.0,None,None,False,NaN,66.0,None
6,19,M-151-0,5,None,0101000020E61000007769C361692244403E5B07077B73...,0.20,77.0,None,None,False,NaN,77.0,None
7,20,M-110-0,5,None,0101000020E6100000AB984A3FE1204440AA0D4E44BFD6...,0.10,88.0,None,None,False,NaN,88.0,None
8,21,M-132-0,5,None,0101000020E6100000ED0F94DBF615444062F6B2EDB455...,0.00,99.0,None,None,False,NaN,99.0,None
9,22,M-153-0,5,None,None,None,100.0,None,None,False,NaN,100.0,None


"M-108-0": 10,
"M-139-0": 11,
"M-43-0": 12,


,Id,Nombre,Fo_Tipoelemento,FechaBaja,TR_Coordenadas,TR_Ocupacion,TR_IntensidadCarrilAsc,TR_VelMedia,TR_Fecha,TR_Sentido_EsAsce,TR_Carriles,TR_IntensidadCarrilDesc,PuntoKm
15,10,M-108-0,5,None,0101000020E610000059501894693244404CFF9254A618...,0.30,6.0,None,None,False,3.0,6.0,None
16,11,M-139-0,5,None,0101000020E610000087359545613144405DC2A1B778D8...,0.30,5.0,None,None,False,3.0,5.0,None
17,12,M-43-0,5,None,0101000020E610000034BF9A03042F44405A4B0169FFA3...,0.20,99.0,None,None,False,3.0,99.0,None


In [3]:
# Obtener todas las filas de HIS_Elemento_Posiciones para los Id de df_elementos

ids = df_elementos["Id"].dropna().astype(int).tolist()

if not ids:
    print("No hay Id en df_elementos.")
else:
    consulta = 'SELECT * FROM public."HIS_Elemento_Posiciones" WHERE "Fo_Elemento" = ANY(%s::int[]) ORDER BY "Fo_Elemento";'
    cursor.execute(consulta, (ids,))
    resultados = cursor.fetchall()
    columnas_pos = [desc[0] for desc in cursor.description]
    df_posiciones = pd.DataFrame(resultados, columns=columnas_pos)
    display(df_posiciones.head(5))


# contamos todas las filas de df que contienen cada Id
print(df_posiciones.value_counts("Fo_Elemento"))

# min max date per Id
print(df_posiciones.groupby("Fo_Elemento")["Fecha"].agg([np.min, np.max]))

# concadenamos la Id y la Fecha para crear un Id_compouesta unico
df_posiciones["Id_compouesta"] = df_posiciones["Fo_Elemento"].astype(str) + "_" + df_posiciones["Fecha"].astype(str)
print(df_posiciones.value_counts("Id_compouesta"))


,Id,Fo_Elemento,Coordenadas,Ocupacion,IntensidadCarrilAsc,VelMedia,Fecha,Sentido_EsAscendente,Carriles,IntensidadCarrilDesc
0,1002139,10,0101000020E610000059501894693244404CFF9254A618...,None,1967.00,None,2027-01-01 00:00:00,None,None,1074.00
1,1002140,10,0101000020E610000059501894693244404CFF9254A618...,None,1128.00,None,2027-01-01 01:00:00,None,None,1606.00
2,1002141,10,0101000020E610000059501894693244404CFF9254A618...,None,582.00,None,2027-01-01 02:00:00,None,None,916.00
3,1002142,10,0101000020E610000059501894693244404CFF9254A618...,None,302.00,None,2027-01-01 03:00:00,None,None,433.00
4,1002143,10,0101000020E610000059501894693244404CFF9254A618...,None,282.00,None,2027-01-01 04:00:00,None,None,274.00


Fo_Elemento
12    8712
11    8688
10    8136
Name: count, dtype: int64
                   min                 max
Fo_Elemento                               
10          2027-01-01 2027-12-31 23:00:00
11          2027-01-01 2027-12-31 23:00:00
12          2027-01-01 2027-12-31 23:00:00
Id_compouesta
12_2027-12-31 07:00:00    1
12_2027-12-31 06:00:00    1
12_2027-12-31 05:00:00    1
12_2027-12-31 04:00:00    1
12_2027-12-31 03:00:00    1
                         ..
10_2027-01-01 04:00:00    1
10_2027-01-01 03:00:00    1
10_2027-01-01 02:00:00    1
10_2027-01-01 01:00:00    1
10_2027-01-01 00:00:00    1
Name: count, Length: 25536, dtype: int64


C:\Users\inaki.campo\AppData\Local\Temp\ipykernel_10268\3731279526.py:20: FutureWarning: The provided callable <function min at 0x0000022DD3131A20> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  print(df_posiciones.groupby("Fo_Elemento")["Fecha"].agg([np.min, np.max]))
C:\Users\inaki.campo\AppData\Local\Temp\ipykernel_10268\3731279526.py:20: FutureWarning: The provided callable <function max at 0x0000022DD3131900> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  print(df_posiciones.groupby("Fo_Elemento")["Fecha"].agg([np.min, np.max]))


In [4]:
# Borramos todas las filas de la tabla his_elemento_posiciones cuyo Fo_Elemento esté en la lista ids y cuya fecha sea anterior a 2027-01-01
# if not ids:
#     print("No hay Id en df_elementos.")
# else:
#     consulta_borrar = 'DELETE FROM public."HIS_Elemento_Posiciones" WHERE "Fo_Elemento" = ANY(%s::int[]) AND "Fecha" < %s;'
#     cursor.execute(consulta_borrar, (ids, '2027-01-01'))
#     conexion.commit()
#     print(f"Se han borrado las filas de HIS_Elemento_Posiciones para los Id {ids} con fecha anterior a 2027-01-01.")


## Cargamos datos 2022 desde excel y agrupamos carriles y sentidos de calzada

In [5]:
# load C:\Users\inaki.campo\Desktop\Sacyr_Ionica\sw-modelo-rutas-trafico\data\datos_sacyr
df_2022 = pd.read_excel("C:/Users/inaki.campo/Desktop/Sacyr_Ionica/sw-modelo-rutas-trafico/data/datos_sacyr/A3_datos_horarios_22.xlsx")
print(df_2022.head())

  Provincia Estación Carretera    Pk Tipo de estación  Calzada  Carril  \
0    MADRID  M-708-0       A-3  5.95       PERMANENTE        1       1   
1    MADRID  M-708-0       A-3  5.95       PERMANENTE        1       1   
2    MADRID  M-708-0       A-3  5.95       PERMANENTE        1       1   
3    MADRID  M-708-0       A-3  5.95       PERMANENTE        1       1   
4    MADRID  M-708-0       A-3  5.95       PERMANENTE        1       1   

                       Dia  Hora  V_Ligeros  V_Pesados  V_Total  
0  2022-01-01 00:00:00.000     0        877          3      880  
1  2022-01-01 00:00:00.000     1        760          5      765  
2  2022-01-01 00:00:00.000     2        386          5      391  
3  2022-01-01 00:00:00.000     3        220          5      225  
4  2022-01-01 00:00:00.000     4        147          4      151  


In [6]:
# Modificamos el df_2022 para insertarlo en la tabla HIS_Elemento_Posiciones

# Columnas:  Fo_Elemento	 Coordenadas	Ocupacion	IntensidadCarrilAsc	VelMedia	Fecha	Sentido_EsAscendente	Carriles	IntensidadCarrilDesc

# Fo_Elemento = 12  # M-108-0
# Coordenadas = POINT (40.39443 -3.63972)
# Ocupacion = Null
# IntensidadCarrilAsc = TO DO
# VelMedia = Null
# Fecha = TO DO
# Sentido_EsAscendente =  ¿¿¿ Borrar ???
# Carriles = ¿¿¿ Borrar ???
# IntensidadCarrilDesc = TO DO

# Unificamos carriles por calzada y pivotamos a columnas Asc/Desc en una sola fila por fecha

# Nos quedamos solo con las estaciones presentes en B_Elementos
df_tmp = df_2022[df_2022["Estación"].isin(df_elementos["Nombre"])].copy()

# Construimos la marca de tiempo# Añadimos 5 años al 2022 para que sea evidente que es de prueba
df_tmp["Dia"] = pd.to_datetime(df_tmp["Dia"], errors="coerce") + pd.DateOffset(years=5)
df_tmp["Hora"] = pd.to_numeric(df_tmp["Hora"], errors="coerce").fillna(0).astype(int)
df_tmp["Fecha"] = df_tmp["Dia"] + pd.to_timedelta(df_tmp["Hora"], unit="h")

# Sumamos los carriles por calzada (dirección) para cada Estación/Fecha
df_sum = (
    df_tmp.groupby(["Estación", "Fecha", "Calzada"], dropna=False, as_index=False)["V_Total"]
    .sum()
)

# Pasamos calzadas a columnas: 1 -> Asc, 2 -> Desc
df_wide = df_sum.pivot(index=["Estación", "Fecha"], columns="Calzada", values="V_Total").reset_index()
#display(df_wide.head(5))
df_wide = df_wide.rename(columns={1: "IntensidadCarrilAsc", 2: "IntensidadCarrilDesc"})
#display(df_wide.head(5))
# Mapeamos Fo_Elemento desde B_Elementos
id_map = df_elementos.set_index("Nombre")["Id"]
df_wide["Fo_Elemento"] = df_wide["Estación"].map(id_map)
#display(df_wide.head(5))

# Rellenamos columnas requeridas
df_wide["Coordenadas"] = None
df_wide["Ocupacion"] = None
df_wide["VelMedia"] = None
for col in ["IntensidadCarrilAsc", "IntensidadCarrilDesc"]:
    if col not in df_wide:
        df_wide[col] = pd.NA

# Reordenamos y tipamos
df_his = df_wide[
    ["Fo_Elemento", "Coordenadas", "Ocupacion", "IntensidadCarrilAsc", "VelMedia", "Fecha", "IntensidadCarrilDesc"]
].copy()

df_his["Fo_Elemento"] = df_his["Fo_Elemento"].astype("Int64")
df_his["IntensidadCarrilAsc"] = df_his["IntensidadCarrilAsc"].astype("Int64")
df_his["IntensidadCarrilDesc"] = df_his["IntensidadCarrilDesc"].astype("Int64")

# Nos quedamos con filas mapeadas a un elemento válido
df_his = df_his[df_his["Fo_Elemento"].notna()].reset_index(drop=True)

# insertmos coordenadas de B_Elementos
coord_map = df_elementos.set_index("Id")["TR_Coordenadas"]
df_his["Coordenadas"] = df_his["Fo_Elemento"].map(coord_map)

display(df_his.head(5))


print(df_his["Fo_Elemento"].unique())

Calzada,Fo_Elemento,Coordenadas,Ocupacion,IntensidadCarrilAsc,VelMedia,Fecha,IntensidadCarrilDesc
0,10,0101000020E610000059501894693244404CFF9254A618...,None,1967,None,2027-01-01 00:00:00,1074
1,10,0101000020E610000059501894693244404CFF9254A618...,None,1128,None,2027-01-01 01:00:00,1606
2,10,0101000020E610000059501894693244404CFF9254A618...,None,582,None,2027-01-01 02:00:00,916
3,10,0101000020E610000059501894693244404CFF9254A618...,None,302,None,2027-01-01 03:00:00,433
4,10,0101000020E610000059501894693244404CFF9254A618...,None,282,None,2027-01-01 04:00:00,274


<IntegerArray>
[10, 11, 12]
Length: 3, dtype: Int64


## Insertamos datos de 2022 para espiras 108 139 y 43 en tabla HIS_Elemento_Posiciones como 2027

In [7]:
from psycopg2.extras import execute_values

## Insertamos los datos en la tabla HIS_Elemento_Posiciones
# Columnas destino en la tabla
cols_destino = (
    '"Fo_Elemento","Coordenadas","Ocupacion",'
    '"IntensidadCarrilAsc","VelMedia","Fecha","IntensidadCarrilDesc"'
)

# Preparar registros con conversión de NA -> None y tipos nativos
registros_insert = [
    (
        int(row.Fo_Elemento) if pd.notna(row.Fo_Elemento) else None,
        row.Coordenadas,
        row.Ocupacion,
        int(row.IntensidadCarrilAsc) if pd.notna(row.IntensidadCarrilAsc) else None,
        row.VelMedia,
        row.Fecha.to_pydatetime() if hasattr(row.Fecha, "to_pydatetime") else row.Fecha,
        int(row.IntensidadCarrilDesc) if pd.notna(row.IntensidadCarrilDesc) else None,
    )
    for _, row in df_his.iterrows()
]

sql_insert = f'INSERT INTO public."HIS_Elemento_Posiciones" ({cols_destino}) VALUES %s'

# try:
#     execute_values(cursor, sql_insert, registros_insert, page_size=1000)
#     conexion.commit()
#     print(f"Filas insertadas: {len(registros_insert)}")
# except Exception as e:
#     conexion.rollback()
#     print(f"Error al insertar: {e}")

### Funciones de creación de input e inferencia de modelos predictivos. 24h y 3L


In [39]:
import holidays
import torch
import os
from omegaconf import OmegaConf
from models import get_model

def crear_input_predicciones_24h(nombre_espira, df_24h):
    print(f"----  Creacion input 24h para {nombre_espira}  -----")
    # fetures dia laborable: Calculamos las fechas del dia objetivo, anterior y posterior
    dia_objetivo = df_24h.iloc[-1]["Fecha"].date()
    print(f"Dia objetivo: {dia_objetivo}")
    dia_posterior = dia_objetivo + pd.Timedelta(days=1)
    print(f"Dia posterior: {dia_posterior}")
    dia_anterior = dia_objetivo - pd.Timedelta(days=1)
    print(f"Dia anterior: {dia_anterior}")

    # Creamos las ventanas de las dos calzadas, 1c es creciente y 2c es decreciente. Formato float32
    ventana_1c = df_24h[["IntensidadCarrilAsc"]].tail(24).T.to_numpy().astype(np.float32)
    ventana_2c = df_24h[["IntensidadCarrilDesc"]].tail(24).T.to_numpy().astype(np.float32)

    md_holidays = holidays.Spain(prov='MD')

    laborable = pd.to_datetime(dia_objetivo).dayofweek < 5
    # Comprobamos si el dia objetivo es laborable para mirar si fue festivo
    if laborable:
        laborable = not (pd.to_datetime(dia_objetivo) in md_holidays)
    new_col = np.array([int(laborable) for _ in range(24)]).reshape(1, 24)
    ventana_1c = np.concatenate((ventana_1c, new_col), axis=0)
    ventana_2c = np.concatenate((ventana_2c, new_col), axis=0)

    # Comprobamos si el dia anterior es laborable
    laborable_anterior = pd.to_datetime(dia_anterior).dayofweek < 5
    if laborable_anterior:
        laborable_anterior = not (pd.to_datetime(dia_anterior) in md_holidays)
    new_col = np.array([int(laborable_anterior) for _ in range(24)]).reshape(1, 24)
    ventana_1c = np.concatenate((ventana_1c, new_col), axis=0)
    ventana_2c = np.concatenate((ventana_2c, new_col), axis=0)

    # Comprobamos si el dia posterior es laborable
    laborable_posterior = pd.to_datetime(dia_posterior).dayofweek < 5
    if laborable_posterior:
        laborable_posterior = not (pd.to_datetime(dia_posterior) in md_holidays)
    new_col = np.array([int(laborable_posterior) for _ in range(24)]).reshape(1, 24)
    ventana_1c = np.concatenate((ventana_1c, new_col), axis=0)
    ventana_2c = np.concatenate((ventana_2c, new_col), axis=0)

    # Convertimos a tensores de torch
    ventana_1c = torch.tensor(ventana_1c).unsqueeze(0)  # shape (1, 4, 24)
    ventana_2c = torch.tensor(ventana_2c).unsqueeze(0)  # shape (1, 4, 24)

    # Hacemos reshape a (1, 24, 4)
    ventana_1c = ventana_1c.permute(0, 2, 1)  # shape (1, 24, 4)
    ventana_2c = ventana_2c.permute(0, 2, 1)  # shape (1, 24, 4)

    # convertimos a float32
    ventana_1c = ventana_1c.float()
    ventana_2c = ventana_2c.float()

    # load model from ./models/{id}.
    print(ventana_1c.shape, ventana_2c.shape)
    print("---- Fin creacion input ----")
    return ventana_1c, ventana_2c

def predicciones_24h(nombre_espira, input_c1, input_c2, device='cuda'):
    print(f"----  Predicciones 24h para {nombre_espira} -----")

    # predicciones 5-fold CV. Cargamos los 5 modelos de la id para la calzada 1
    preds_c1 = []
    preds_c2 = []
    for fold in range(5):
        # model path absoluto del fichero, cuya ubicacion desconocemos
        this_dir = os.getcwd()
        # this_dir = os.path.dirname(__file__)
        model_path_c1 = os.path.join(this_dir, "models", f"{str(nombre_espira)}-1c", f"best_{fold}.pth")
        model_path_c2 = os.path.join(this_dir, "models", f"{str(nombre_espira)}-2c", f"best_{fold}.pth")
        print(f"Model path C1: {model_path_c1}")
        print(f"Model path C2: {model_path_c2}")
        # load models config
        config_path_c1 = os.path.join(this_dir, "models", f"{str(nombre_espira)}-1c", "config.yaml")
        config_path_c2 = os.path.join(this_dir, "models", f"{str(nombre_espira)}-2c", "config.yaml")
        config_c1 = OmegaConf.load(config_path_c1)
        config_c2 = OmegaConf.load(config_path_c2)
        print(config_c1)
        print(config_c2)
        # Instanciamos los objetos modelo
        input_size, output_size = input_c1[0].shape, 24
        print(f"Input size: {input_size}, Output size: {output_size}")
        model_1 = get_model(input_size, output_size, config_c1.model)
        model_2 = get_model(input_size, output_size, config_c2.model)
        model_1.to(device)
        model_2.to(device)
        input_c1 = input_c1.to(device)
        input_c2 = input_c2.to(device)
        print(f"Input C1 device: {input_c1.device}, Input C2 device: {input_c2.device}")
        # cargamos checkpint en el modelo
        load_checkpoint(model_1, model_path_c1, device)
        load_checkpoint(model_2, model_path_c2, device)
        model_1.eval()
        model_2.eval()
        # inferencia
        with torch.no_grad():
            pred_1 = model_1(input_c1).cpu().squeeze().numpy()  # shape (24,)
            pred_2 = model_2(input_c2).cpu().squeeze().numpy()  # shape (24,)
            preds_c1.append(pred_1)
            preds_c2.append(pred_2)
    # media de las predicciones
    pred_c1_mean = np.mean(preds_c1, axis=0)
    pred_c2_mean = np.mean(preds_c2, axis=0)
    print(f"Predicción sin desescalar calzada 1c (media 5 folds): {pred_c1_mean}")
    print(f"Predicción sin desescalar calzada 2c (media 5 folds): {pred_c2_mean}")
    print("-------- Fin predicciones ---------")
    return pred_c1_mean, pred_c2_mean


def load_checkpoint(model, checkpoint_path, device):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    if "model_state" in checkpoint:
        state_dict = checkpoint["model_state"]
    else:
        state_dict = checkpoint

    model.load_state_dict(state_dict)


def compute_trasvase(row, trafico_minimo=1500, factor_mejora_minimo=1.3):
    # if both flows are lower than 1500 then no trasvase
    if row['IntensidadCarrilAsc'] < trafico_minimo and row['IntensidadCarrilDesc'] < trafico_minimo:
        return False, False, 0, 0
    
    asc_is_bigger = row['IntensidadCarrilAsc'] >= row['IntensidadCarrilDesc']

    flujo_medio = (row['IntensidadCarrilAsc'] + row['IntensidadCarrilDesc']) / (row['n_carriles'] * 2)

    # identify the calzada with higher and lower flow
    if row['IntensidadCarrilAsc'] >= row['IntensidadCarrilDesc']:
        high_flow = row['IntensidadCarrilAsc']
        low_flow = row['IntensidadCarrilDesc']
        n_high = row['n_carriles']
        n_low = row['n_carriles']
    else:
        high_flow = row['IntensidadCarrilDesc']
        low_flow = row['IntensidadCarrilAsc']
        n_high = row['n_carriles']
        n_low = row['n_carriles']

    # if n_low < 2 then no trasvase
    if n_low < 2:
        return False, False, 0, 0
    
    # original average per lane using original number of carriles
    orig_avg_high = high_flow / n_high
    orig_avg_low = low_flow / n_low
    original_avg_error = abs(flujo_medio - orig_avg_high)*n_high + abs(flujo_medio - orig_avg_low)*n_low

    modified_high = high_flow / (n_high + 1)
    modified_low = low_flow / (n_low - 1)
    modified_avg_error = abs(flujo_medio - modified_high)*(n_high + 1) + abs(flujo_medio - modified_low)*(n_low - 1)
    
    # if the modified average error is lower than the original average error then we will do the trasvase
    if modified_avg_error < original_avg_error * (1 / factor_mejora_minimo):
        if asc_is_bigger:
            new_asc_flow = modified_high * n_high
            new_desc_flow = modified_low * (n_low - 1) + modified_high * 1
            return True, False, new_asc_flow, new_desc_flow
        else:
            new_desc_flow = modified_high * n_high
            new_asc_flow = modified_low * (n_low - 1) + modified_high * 1
            return False, True, new_asc_flow, new_desc_flow
    else:
        return False, False, 0, 0



### Carga ultimas 24h por cada espira, inferencia

In [40]:
# fecha_actual = pd.Timestamp.now()
fecha_actual = pd.to_datetime("2027-01-03 23:00:00")
print(f"Fecha actual: {fecha_actual}")

trafico_minimo=1500
factor_mejora_minimo=1.3
print(f"Tráfico mínimo: {trafico_minimo}, Factor mejora mínimo: {factor_mejora_minimo}")

predicciones_espiras = {}
df_posiciones["Fecha"] = pd.to_datetime(df_posiciones["Fecha"], errors="coerce")
# seleccionamos las ultimas 48h registradas para cada Id de Fo_Elemento en df_posiciones
for Id, nombre_espira, n_carriles in df_elementos[["Id", "Nombre", "TR_Carriles"]].values:
    print(f"--- Creando Datos Input para espira {nombre_espira}  ID = {Id} ---")
    df_tmp = df_posiciones[df_posiciones["Fo_Elemento"] == Id].copy()
    # float
    df_tmp[["IntensidadCarrilAsc", "IntensidadCarrilDesc"]] = df_tmp[["IntensidadCarrilAsc", "IntensidadCarrilDesc"]].astype(np.float32)
    # Hacemos un escalado standard scaler de las columnas IntensidadCarrilAsc y IntensidadCarrilDesc
    medias = df_tmp[["IntensidadCarrilAsc", "IntensidadCarrilDesc"]].mean()
    stds = df_tmp[["IntensidadCarrilAsc", "IntensidadCarrilDesc"]].std()
    df_tmp["IntensidadCarrilAsc"] = (df_tmp["IntensidadCarrilAsc"] - medias["IntensidadCarrilAsc"]) / stds["IntensidadCarrilAsc"]
    df_tmp["IntensidadCarrilDesc"] = (df_tmp["IntensidadCarrilDesc"] - medias["IntensidadCarrilDesc"]) / stds["IntensidadCarrilDesc"]   
    print(f"Media y std de IntensidadCarrilAsc: {medias['IntensidadCarrilAsc']}, {stds['IntensidadCarrilAsc']}")
    print(f"Media y std de IntensidadCarrilDesc: {medias['IntensidadCarrilDesc']}, {stds['IntensidadCarrilDesc']}")

    if not df_tmp.empty:
        min_fecha = fecha_actual - pd.Timedelta(hours=48)
        df_48h = df_tmp[(df_tmp["Fecha"] > min_fecha) & (df_tmp["Fecha"] <= fecha_actual)]
        print(f"{nombre_espira}  Id: {Id}, Registros últimas 48h: {len(df_48h)}")

        # Por cada df, cogemos las ultimas 24 medidas (si hay tantas)
        if df_48h.shape[0] > 24:
            df_24h = df_48h.nlargest(24, "Fecha")
            df_24h = df_24h[["Fecha", "IntensidadCarrilAsc", "IntensidadCarrilDesc"]]
            print(df_24h.head(3))
            min_fecha_24h = df_24h["Fecha"].min()
            max_fecha_24h = df_24h["Fecha"].max()
            print(f"{nombre_espira}  Id: {Id}, Min Fecha 24h: {min_fecha_24h}, Max Fecha 24h: {max_fecha_24h}")
            print(f"Procedemos a predecir las próximas 24h para Id: {Id}")
            horas_a_predecir = pd.date_range(start=max_fecha_24h + pd.Timedelta(hours=1), periods=24, freq='h')
            print(f"horas a predecir: {horas_a_predecir[0]} - {horas_a_predecir[-1]}")

            input_c1, input_c2 = crear_input_predicciones_24h(nombre_espira, df_24h)

            pred_c1, pred_c2 = predicciones_24h(nombre_espira, input_c1, input_c2)
            
            # desescalamos las predicciones
            pred_c1 = (pred_c1 * stds["IntensidadCarrilAsc"]) + medias["IntensidadCarrilAsc"]
            pred_c2 = (pred_c2 * stds["IntensidadCarrilDesc"]) + medias["IntensidadCarrilDesc"]
            print(f"Predicción desescalada calzada 1c (media 5 folds): {pred_c1}")
            print(f"Predicción desescalada calzada 2c (media 5 folds): {pred_c2}")
            # Creamos un df con las predicciones y las horas a predecir
            df_preds = pd.DataFrame({
                "Fecha": horas_a_predecir,
                "IntensidadCarrilAsc": pred_c1,
                "IntensidadCarrilDesc": pred_c2,
                "n_carriles": n_carriles,
                "Fo_Elemento": Id
            })
            df_preds[['AbrirCarrilAsc', 'AbrirCarrilDesc', 'IntensidadPredCarrilAsc', 'IntensidadPredCarrilDesc']] = df_preds.apply(lambda row: compute_trasvase(row, trafico_minimo, factor_mejora_minimo), axis=1,  result_type='expand')

            display(df_preds.head(3))

            # guardamos las predicciones en el diccionario con clave Id-1c y Id-2c
            predicciones_espiras[f"{Id}"] = df_preds

            print("--------------------------------------------------")

        else:
            print(f"Id: {Id} no tiene suficientes registros en las últimas 48h para predicciones (tiene {df_48h.shape[0]}).")
        
    else:
        print(f"Id: {Id} no tiene registros en df_posiciones.")


Fecha actual: 2027-01-03 23:00:00
Tráfico mínimo: 1500, Factor mejora mínimo: 1.3
--- Creando Datos Input para espira M-108-0  ID = 10 ---
Media y std de IntensidadCarrilAsc: 1848.4244384765625, 1143.395751953125
Media y std de IntensidadCarrilDesc: 2111.9599609375, 1370.5921630859375
M-108-0  Id: 10, Registros últimas 48h: 48
                 Fecha  IntensidadCarrilAsc  IntensidadCarrilDesc
71 2027-01-03 23:00:00            -1.205553             -1.086362
70 2027-01-03 22:00:00            -0.900322             -0.729582
69 2027-01-03 21:00:00            -0.240008             -0.386665
M-108-0  Id: 10, Min Fecha 24h: 2027-01-03 00:00:00, Max Fecha 24h: 2027-01-03 23:00:00
Procedemos a predecir las próximas 24h para Id: 10
horas a predecir: 2027-01-04 00:00:00 - 2027-01-04 23:00:00
----  Creacion input 24h para M-108-0  -----
Dia objetivo: 2027-01-03
Dia posterior: 2027-01-04
Dia anterior: 2027-01-02
torch.Size([1, 24, 4]) torch.Size([1, 24, 4])
---- Fin creacion input ----
----  Predic

c:\Users\inaki.campo\AppData\Local\miniconda3\envs\sacyr\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3607113742925706 and num_layers=1
  warnings.warn(
c:\Users\inaki.campo\AppData\Local\miniconda3\envs\sacyr\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1529558592446795 and num_layers=1
  warnings.warn(


Input C1 device: cuda:0, Input C2 device: cuda:0
Model path C1: c:\Users\inaki.campo\Desktop\Sacyr_Ionica\service\models\M-108-0-1c\best_1.pth
Model path C2: c:\Users\inaki.campo\Desktop\Sacyr_Ionica\service\models\M-108-0-2c\best_1.pth
{'dataset': {'dataset_module': 'sacyr', 'root': './data/datasets', 'num_workers': 4, 'mode': 'reggresion', 'kfold': {'num_folds': 5, 'shuffle': True}, 'name': '108-24h-24h-3L-1c'}, 'scaler': {'name': 'standard'}, 'loss': {'name': 'mse'}, 'optimizer': {'name': 'adam', 'lr': 0.001, 'weight_decay': 0.0001, 'scheduler': 'step', 'step_size': 10, 'gamma': 0.1}, 'training': {'epochs': 100, 'batch_size': 1024, 'num_workers': 2, 'prefetch_factor': 3, 'seed': 42, 'device': 'cuda', 'log_dir': 'logs'}, 'scheduler': {'name': 'reduce_on_plateau', 'patience': 5, 'factor': 0.1}, 'metrics': [{'name': 'mae-sacyr-1h', 'prediction_horizon': 1}], 'callbacks': [{'name': 'checkpoint', 'dirpath': 'checkpoints/', 'monitor': 'Val_loss', 'mode': 'min'}, {'name': 'wandb_logger', '

,Fecha,IntensidadCarrilAsc,IntensidadCarrilDesc,n_carriles,Fo_Elemento,AbrirCarrilAsc,AbrirCarrilDesc,IntensidadPredCarrilAsc,IntensidadPredCarrilDesc
0,2027-01-04 00:00:00,737.069824,374.643433,3.0,10,False,False,0.000000,0.000000
1,2027-01-04 01:00:00,1498.639893,1241.818726,3.0,10,False,False,0.000000,0.000000
2,2027-01-04 02:00:00,1738.524658,2798.626953,3.0,10,False,True,2438.181396,2098.970215


--------------------------------------------------
--- Creando Datos Input para espira M-139-0  ID = 11 ---
Media y std de IntensidadCarrilAsc: 3504.16943359375, 2107.03955078125
Media y std de IntensidadCarrilDesc: 3236.4453125, 2001.6129150390625
M-139-0  Id: 11, Registros últimas 48h: 48
                   Fecha  IntensidadCarrilAsc  IntensidadCarrilDesc
8207 2027-01-03 23:00:00            -1.409166             -1.432567
8206 2027-01-03 22:00:00            -1.170918             -1.319159
8205 2027-01-03 21:00:00            -0.792662             -1.013405
M-139-0  Id: 11, Min Fecha 24h: 2027-01-03 00:00:00, Max Fecha 24h: 2027-01-03 23:00:00
Procedemos a predecir las próximas 24h para Id: 11
horas a predecir: 2027-01-04 00:00:00 - 2027-01-04 23:00:00
----  Creacion input 24h para M-139-0  -----
Dia objetivo: 2027-01-03
Dia posterior: 2027-01-04
Dia anterior: 2027-01-02
torch.Size([1, 24, 4]) torch.Size([1, 24, 4])
---- Fin creacion input ----
----  Predicciones 24h para M-139-0 -----

...

--------------------------------------------------
--- Creando Datos Input para espira M-43-0  ID = 12 ---
Media y std de IntensidadCarrilAsc: 1322.923095703125, 758.581787109375
Media y std de IntensidadCarrilDesc: 1748.6697998046875, 1070.0205078125
M-43-0  Id: 12, Registros últimas 48h: 48
                    Fecha  IntensidadCarrilAsc  IntensidadCarrilDesc
16895 2027-01-03 23:00:00            -1.138866             -1.461346
16894 2027-01-03 22:00:00            -0.454695             -1.317423
16893 2027-01-03 21:00:00            -0.055265             -1.051073
M-43-0  Id: 12, Min Fecha 24h: 2027-01-03 00:00:00, Max Fecha 24h: 2027-01-03 23:00:00
Procedemos a predecir las próximas 24h para Id: 12
horas a predecir: 2027-01-04 00:00:00 - 2027-01-04 23:00:00
----  Creacion input 24h para M-43-0  -----
Dia objetivo: 2027-01-03
Dia posterior: 2027-01-04
Dia anterior: 2027-01-02
torch.Size([1, 24, 4]) torch.Size([1, 24, 4])
---- Fin creacion input ----
----  Predicciones 24h para M-43-0 --

c:\Users\inaki.campo\AppData\Local\miniconda3\envs\sacyr\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.14525356972625547 and num_layers=1
  warnings.warn(
c:\Users\inaki.campo\AppData\Local\miniconda3\envs\sacyr\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.22417876163845044 and num_layers=1
  warnings.warn(


Building CNN-LSTM model with config: {'cnn_filters': 153, 'dropout': 0.22417876163845044, 'hidden_size': 417, 'kernel_size': 2, 'name': 'cnn_lstm', 'num_layers': 1}
cnn_lstm module:  input_size: torch.Size([24, 4]), output_size: 24, config: {'cnn_filters': 153, 'dropout': 0.22417876163845044, 'hidden_size': 417, 'kernel_size': 2, 'name': 'cnn_lstm', 'num_layers': 1}
Input C1 device: cuda:0, Input C2 device: cuda:0
Model path C1: c:\Users\inaki.campo\Desktop\Sacyr_Ionica\service\models\M-43-0-1c\best_2.pth
Model path C2: c:\Users\inaki.campo\Desktop\Sacyr_Ionica\service\models\M-43-0-2c\best_2.pth
{'dataset': {'dataset_module': 'sacyr', 'root': './data/datasets', 'num_workers': 4, 'mode': 'reggresion', 'kfold': {'num_folds': 5, 'shuffle': True}, 'name': '43-24h-24h-3L-1c'}, 'scaler': {'name': 'standard'}, 'loss': {'name': 'mse'}, 'optimizer': {'name': 'adam', 'lr': 0.001, 'weight_decay': 0.0001, 'scheduler': 'step', 'step_size': 10, 'gamma': 0.1}, 'training': {'epochs': 100, 'batch_size

,Fecha,IntensidadCarrilAsc,IntensidadCarrilDesc,n_carriles,Fo_Elemento,AbrirCarrilAsc,AbrirCarrilDesc,IntensidadPredCarrilAsc,IntensidadPredCarrilDesc
0,2027-01-04 00:00:00,649.033997,116.396606,3.0,12,False,False,0.00000,0.000000
1,2027-01-04 01:00:00,1253.619141,282.303833,3.0,12,False,False,0.00000,0.000000
2,2027-01-04 02:00:00,1727.621094,758.980530,3.0,12,True,False,1295.71582,1190.885803


--------------------------------------------------


### Insertamos las predicciones en la tabla 

In [44]:
conexion = psycopg2.connect(
    host="srvbu-bd",
    port="5432",
    database="BITAL_SACYR_ICONICA",
    user="adminpg",
    password="KRZEy5fxQyNmmVjeUseQ",
)

cursor = conexion.cursor()

for k, v in predicciones_espiras.items():
    # insert into B_PrediccionTraficoe
    # Preparamos e insertamos (omitimos n_carriles)
    cols_sql = '"Fo_Elemento","Fecha","IntensidadCarrilAsc","IntensidadCarrilDesc","AbrirCarrilAsc","AbrirCarrilDesc","IntensidadPredCarrilAsc","IntensidadPredCarrilDesc"'
    registros = [
        (
            int(r.Fo_Elemento),
            r.Fecha.to_pydatetime(),
            float(r.IntensidadCarrilAsc),
            float(r.IntensidadCarrilDesc),
            bool(r.AbrirCarrilAsc),
            bool(r.AbrirCarrilDesc),
            float(r.IntensidadPredCarrilAsc),
            float(r.IntensidadPredCarrilDesc),
        )
        for r in v.itertuples(index=False)
    ]
    if registros:
        sql = f'INSERT INTO public."B_PrediccionTrafico" ({cols_sql}) VALUES %s'
        try:
            execute_values(cursor, sql, registros, page_size=100)
            conexion.commit()
            print(f"Insertadas {len(registros)} filas para Fo_Elemento {k}")
        except Exception as e:
            conexion.rollback()
            print(f"Error insertando Fo_Elemento {k}: {e}")
    

Error insertando Fo_Elemento 10: llave duplicada viola restricción de unicidad «uq_elemento_fecha»
DETAIL:  Ya existe la llave ("Fo_Elemento", "Fecha")=(10, 2027-01-04 00:00:00).

Error insertando Fo_Elemento 11: llave duplicada viola restricción de unicidad «uq_elemento_fecha»
DETAIL:  Ya existe la llave ("Fo_Elemento", "Fecha")=(11, 2027-01-04 00:00:00).

Error insertando Fo_Elemento 12: llave duplicada viola restricción de unicidad «uq_elemento_fecha»
DETAIL:  Ya existe la llave ("Fo_Elemento", "Fecha")=(12, 2027-01-04 00:00:00).

